In [1]:
# Load Dataset from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import Libraries
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

train_file_path = '/content/drive/MyDrive/IndiaAI_Datasets/preprocessed_test_data.csv'
test_file_path= '/content/drive/MyDrive/IndiaAI_Datasets/preprocessed_test_data.csv'


train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

# Use 'lemmatized_complaints' as the text column and 'category' as the label column
train_texts, train_labels = train_df['lemmatized_complaints'].tolist(), train_df['category'].tolist()
val_texts, val_labels = test_df['lemmatized_complaints'].tolist(), test_df['category'].tolist()

# Initialize Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert text data to strings and handle any missing values
train_texts = train_df['lemmatized_complaints'].fillna("").astype(str).tolist()
val_texts = test_df['lemmatized_complaints'].fillna("").astype(str).tolist()
train_labels = train_df['category'].tolist()
val_labels = test_df['category'].tolist()

# Create label mapping
labels_mapping = {label: idx for idx, label in enumerate(train_df['category'].unique())}
train_labels = [labels_mapping[label] for label in train_df['category']]
val_labels = [labels_mapping[label] for label in test_df['category']]

import json

# Create the label mapping during training
labels_mapping = {label: idx for idx, label in enumerate(train_df['category'].unique())}

# Define the path to save the mapping
mapping_save_path = '/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_model/labels_mapping.json'

# Save the mapping as a JSON file
with open(mapping_save_path, 'w') as f:
    json.dump(labels_mapping, f)

print(f"Label mapping saved to {mapping_save_path}")


# Convert text data to strings and handle any missing values
train_texts = train_df['lemmatized_complaints'].fillna("").astype(str).tolist()
val_texts = test_df['lemmatized_complaints'].fillna("").astype(str).tolist()


# Tokenize Texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Label mapping saved to /content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_model/labels_mapping.json


KeyboardInterrupt: 

In [ ]:
# # Create Dataset Class
# class CyberDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item
#     def __len__(self):
#         return len(self.labels)

# train_dataset = CyberDataset(train_encodings, train_labels)
# val_dataset = CyberDataset(val_encodings, val_labels)

# # Initialize BERT Model
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(train_labels)))

# #previous training for accuracy 0.84
# # # Set Up Training Arguments and Trainer
# # training_args = TrainingArguments(
# #     output_dir='./results',
# #     evaluation_strategy="epoch",
# #     per_device_train_batch_size=8,
# #     per_device_eval_batch_size=8,
# #     num_train_epochs=3,
# #     weight_decay=0.01
# # )

# # Set Up Training Arguments and Trainer with Hyperparameter Tuning
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy="epoch",
#     save_strategy="epoch",  # Save at the end of each epoch
#     save_total_limit=2,      # Keep only the last 2 checkpoints
#     per_device_train_batch_size=8,  # Smaller batch size for better generalization
#     per_device_eval_batch_size=8,     # Smaller batch size for evaluation
#     num_train_epochs=3,               # Increased epochs for more training
#     weight_decay=0.01,
#     learning_rate=2e-5,               # Learning rate between 1e-5 and 3e-5
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset
# )


In [ ]:
import torch
from transformers import TrainingArguments, Trainer, BertForSequenceClassification
from pathlib import Path
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the checkpoint path
checkpoint_path = '/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_checkpoint.pth'

# Create Dataset Class
class CyberDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# Initialize datasets
train_dataset = CyberDataset(train_encodings, train_labels)
val_dataset = CyberDataset(val_encodings, val_labels)

# Initialize BERT Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(train_labels)))

# Define Training Arguments without W&B
training_args = TrainingArguments(
    output_dir='./results',            # Directory for intermediate checkpoints
    evaluation_strategy="epoch",       # Evaluate at the end of each epoch
    per_device_train_batch_size=16,     # Training batch size
    per_device_eval_batch_size=16,      # Evaluation batch size
    num_train_epochs=2,                # Number of epochs
    weight_decay=0.01,                 # Weight decay
    learning_rate=3e-5,                # Learning rate
    logging_dir='./logs',              # Directory for logs
    logging_steps=100,                 # Log every 100 steps
    save_total_limit=1,                # Only keep the most recent checkpoint
    report_to="none"                   # Disable W&B logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Function to save a manual checkpoint to Google Drive
def save_checkpoint_to_drive(trainer, path):
    checkpoint = {
        "model_state_dict": trainer.model.state_dict(),
        "optimizer_state_dict": trainer.optimizer.state_dict() if trainer.optimizer else None,
        "scheduler_state_dict": trainer.lr_scheduler.state_dict() if trainer.lr_scheduler else None,
        "epoch": trainer.state.epoch,
        "step": trainer.state.global_step
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved to {path}")

# Resume from a checkpoint if it exists
def load_checkpoint_from_drive(model, path):
    checkpoint_file = Path(path)
    if checkpoint_file.exists():
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint["model_state_dict"])
        print(f"Checkpoint loaded from {path}, resuming from epoch {checkpoint['epoch'] + 1}, step {checkpoint['step']}")
        return checkpoint["epoch"], checkpoint["step"]
    print("No checkpoint found, starting training from scratch.")
    return 0, 0

# Load checkpoint from Google Drive if it exists
start_epoch, start_step = load_checkpoint_from_drive(model, checkpoint_path)

# Custom training loop to save checkpoints on Google Drive
epochs = 2  # Number of epochs to train
for epoch in range(int(start_epoch), epochs):
    trainer.state.epoch = epoch  # Update trainer's epoch
    print(f"Starting Epoch {epoch + 1}/{epochs}")

    # Train for one epoch
    trainer.train(resume_from_checkpoint=None)  # Resume training from internal checkpointing

    # Save checkpoint to Google Drive at the end of the epoch
    save_checkpoint_to_drive(trainer, checkpoint_path)

print("Training completed.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


No checkpoint found, starting training from scratch.
Starting Epoch 1/2


Epoch,Training Loss,Validation Loss
1,0.621500,0.532724
2,0.484400,0.456287


Checkpoint saved to /content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_checkpoint.pth
Starting Epoch 2/2


Epoch,Training Loss,Validation Loss
1,0.553400,0.402610
2,0.392600,0.277563


Checkpoint saved to /content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_checkpoint.pth
Training completed.


In [ ]:
# # Train the Model
# trainer.train()

In [ ]:
model.save_pretrained('./v2_bert-base uncased')
print("Model  saved!")
model.save_pretrained('/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_model')
tokenizer.save_pretrained('/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_tokenizer')
# model.save_pretrained('/content/drive/MyDrive/IndianAI/v2_bert_model')
# tokenizer.save_pretrained('/content/drive/MyDrive/IndianAI/v2_bert_tokenizer')


Model  saved!


('/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_tokenizer/vocab.txt',
 '/content/drive/MyDrive/IndiaAI_Datasets/v2_subcat_bert_tokenizer/added_tokens.json')

In [ ]:
# Move model to the same device as the inputs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Make Predictions
test_texts = ["My computer system has been locked with a message asking for payment in cryptocurrency to regain access"]  # Replace with actual complaint text
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings = {key: val.to(device) for key, val in test_encodings.items()}  # Move inputs to the same device

# Run the model
outputs = model(**test_encodings)
predictions = torch.argmax(outputs.logits, dim=1)
print("Predicted labels:", predictions.cpu().numpy())  # Move predictions back to CPU for readability


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Update trainer to include custom metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Run evaluation
eval_results = trainer.evaluate()

# Print detailed results
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall: {eval_results['eval_recall']:.4f}")
print(f"F1 Score: {eval_results['eval_f1']:.4f}")


Accuracy: 0.9101
Precision: 0.9078
Recall: 0.9101
F1 Score: 0.9049


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# to load the saved model
from transformers import AutoModel

# Load the saved model
model = AutoModel.from_pretrained('./bert-base-uncased')
print("Model loaded successfully!")
